Getting consensus sequences from BOLD
====================

Use this python notebook to automatically get a consensus sequence for your taxa of interest from the BOLD db. 

To run, you'll need a python environment with scikit bio 0.2.3 (the version installed with Qiime 1.9.x) and the requests library for using RESTful APIs. To install this environment, you can use the following shell commands to download and install miniconda:

    wget https://repo.continuum.io/miniconda/Miniconda-latest-Linux-x86_64.sh
    chmod +x Miniconda-latest-Linux-x86_64.sh
    ./Miniconda-latest-Linux-x86_64.sh -b
    export PATH=${HOME}/miniconda/bin/:$PATH

    conda create -n BOLD_get pip requests scikit-bio==0.2.3 notebook
    source activate BOLD_get

In [1]:
import requests
import subprocess
import re

from skbio.alignment import Alignment, SequenceCollection
from skbio.parse.sequences import parse_fasta
from skbio.sequence import DNA, DNASequence
from StringIO import StringIO
from skbio.format.sequences import fasta_from_sequences

In [2]:
# Align with muscle

def muscle_align(fasta, muscle_x = 'muscle', muscle_params = []):
    cmd = [muscle_x] #+ [muscle_params]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE,
                              stdin=subprocess.PIPE)
    out, err = p.communicate(fasta)
    return out, err


In [3]:
def terminal_degap(seq, term_char='.', gap_char='-'):
    new_seq = list(seq)
    i = 0
    while new_seq[i] == gap_char:
        new_seq[i] = term_char
        i += 1
    i = -1
    while new_seq[i] == gap_char:
        new_seq[i] = term_char
        i -= 1
    return(''.join(new_seq))

def terminal_degap_fasta_f(fasta_f):
    fasta_records = parse_fasta(fasta_f)
    
    new_aln_seqs = []
    
    for label, seq in fasta_records:
        seq = terminal_degap(seq)
        new_aln_seqs += [DNASequence(seq,label)]
    new_aln_fasta = fasta_from_sequences(new_aln_seqs)
    return(StringIO(new_aln_fasta))


In [4]:
def renorm_dict(d, exclude):
    b = set(d.keys()) - set(exclude)
    e = {k: d[k] for k in b} 
    total = float(sum(e.values()))
    f = {k: float(e[k])/total for k in e}
    
    return(f)

In [5]:
def iupac_sub(nucs):
    IUPAC_lookup = {
        'R': ['A', 'G'],
        'Y': ['C', 'T'],
        'S': ['G', 'C'],
        'W': ['A', 'T'],
        'K': ['G', 'T'],
        'M': ['A', 'C'],
        'B': ['C', 'G', 'T'],
        'D': ['A', 'G', 'T'],
        'H': ['A', 'C', 'T'],
        'V': ['A', 'C', 'G'],
        'N': ['A', 'C', 'G', 'T']
    }
    nucs = [x.upper() for x in nucs]
    for key in IUPAC_lookup:
        if set(nucs) == set(IUPAC_lookup[key]):
            return(key)
    
    return('N')

In [6]:
def simple_consensus(aln, min_freq=0.5, ignore_gap_chars=['.'], degenerate=False):
    pf = aln.position_frequencies()
    consensus_seq = ['*']*aln.sequence_length()
    consensus_freq = ['*']*aln.sequence_length()
    i = 0
    for pos in pf:
        # renormalize freqs after getting rid of ignored (terminal) gaps
        pf_i = renorm_dict(pos, ignore_gap_chars)
        
        # get max freq
        max_freq = max(pf_i.values())
        consensus_freq[i] = max_freq
        
        n_max = [key for key,val in pf_i.iteritems() if val == max_freq]
        
        if max_freq > min_freq and len(n_max) == 1:
            consensus_seq[i] = n_max[0]
        elif max_freq > min_freq and len(n_max) > 1 and degenerate:
            consensus_seq[i] = iupac_sub[n_max]
        elif degenerate:
            consensus_seq[i] = iupac_sub[pf_i.keys()]
        else:
            consensus_seq[i] = 'N'
        i += 1
    
    return(''.join(consensus_seq),consensus_freq)

In [7]:
def limit_records(seqs, n = 10, limit = 'longest'):
    if limit == 'longest':
        ids = seqs.ids()
        lengths = seqs.sequence_lengths()
        
        sorted_ids = [x for (y,x) in sorted(zip(lengths,ids), key=lambda pair: pair[0], reverse=True)]
        
        ids_to_keep = sorted_ids[0:n]
        
    if limit == 'first':
        ids = seqs.ids()
        ids_to_keep = ids[0:n]
    
    limited_seqs = SequenceCollection([seqs[x] for x in ids_to_keep])
    
    return(limited_seqs)

In [8]:
def get_BOLD_consensus_seq(taxon, marker, max_seqs=40, limit='longest', BOLD_url = 'http://www.boldsystems.org/index.php/API_Public/sequence'):
    # get sequences
    try:
        r = requests.get(BOLD_url, params={'taxon':taxon, 'marker':marker}, timeout=5)
    except  requests.exceptions.Timeout as e:
        print(e, 'Request timed out; does taxon exist?')
        return(taxon, None, None)
    except requests.exceptions.ConnectionError as e:
        print(e, 'Request timed out; does taxon exist?')
        return(taxon, None, None)      
    seqs = SequenceCollection.from_fasta_records(parse_fasta(StringIO(r.text), label_to_name = lambda x: re.sub (' ', '_', x)),DNA)
    
    seqs = limit_records(seqs, n = max_seqs, limit = limit)
    
    # generate alignment
    m_aln, m_err = muscle_align(seqs.to_fasta())
    
    # degap alignment
    muscle_aln_degap_f = terminal_degap_fasta_f(StringIO(m_aln))

    muscle_aln_degapped = Alignment.from_fasta_records(parse_fasta(muscle_aln_degap_f),DNA)

    # make consensus sequences
    consensus_seq, consensus_freq = simple_consensus(muscle_aln_degapped)
    
    # return fasta pair
    return('{0} {1} consensus sequence retrieved from BOLD database'.format(re.sub(' ','_',taxon), marker),consensus_seq,seqs.to_fasta())


In [9]:
species_list = ['Halictus crenicornis', 'Halictus fulvipes', 'Halictus fulvipes', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus maculatus', 'Halictus pollinosus', 'Halictus pollinosus', 'Halictus rubicundus', 'Halictus rubicundus', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus scabiosae', 'Halictus Halictus_sp', 'Halictus tumulorum', 'Halictus tumulorum', 'Halictus tumulorum', 'Halictus tumulorum', 'Halictus tumulorum', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum albipes', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum calceatum', 'Lasioglossum fulvicorne', 'Lasioglossum interruptum', 'Lasioglossum interruptum', 'Lasioglossum laevigatum', 'Lasioglossum laevigatum', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum laticeps', 'Lasioglossum lativentre', 'Lasioglossum lativentre', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum leucozonium', 'Lasioglossum limbellum', 'Lasioglossum limbellum', 'Lasioglossum limbellum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum malachurum', 'Lasioglossum marginatum', 'Lasioglossum mediterraneum', 'Lasioglossum morio', 'Lasioglossum morio', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum nigripes', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum pauxillum', 'Lasioglossum villosulum', 'Lasioglossum villosulum', 'Lasioglossum villosulum', 'Lasioglossum villosulum', 'Lasioglossum villosulum', 'Lasioglossum villosulum', 'Lasioglossum zonulum', 'Sphecodes geoffrellus', 'Sphecodes Sphecodes_sp', 'Sphecodes Sphecodes_sp', 'Sphecodes Sphecodes_sp', 'Sphecodes Sphecodes_sp', 'Sphecodes Sphecodes_sp']

species = set(species_list)

species

{'Halictus Halictus_sp',
 'Halictus crenicornis',
 'Halictus fulvipes',
 'Halictus maculatus',
 'Halictus pollinosus',
 'Halictus rubicundus',
 'Halictus scabiosae',
 'Halictus tumulorum',
 'Lasioglossum albipes',
 'Lasioglossum calceatum',
 'Lasioglossum fulvicorne',
 'Lasioglossum interruptum',
 'Lasioglossum laevigatum',
 'Lasioglossum laticeps',
 'Lasioglossum lativentre',
 'Lasioglossum leucozonium',
 'Lasioglossum limbellum',
 'Lasioglossum malachurum',
 'Lasioglossum marginatum',
 'Lasioglossum mediterraneum',
 'Lasioglossum morio',
 'Lasioglossum nigripes',
 'Lasioglossum pauxillum',
 'Lasioglossum villosulum',
 'Lasioglossum zonulum',
 'Sphecodes Sphecodes_sp',
 'Sphecodes geoffrellus'}

In [10]:
marker = 'COI-5P'
headers = []
seqs = []

for taxon in species:
    print(taxon)
    header, seq, fastas = get_BOLD_consensus_seq(taxon, marker)
    
    if seq is None:
        continue
    
    headers += [header]
    seqs += [seq]
    
    with open('{0}.{1}.fasta'.format(re.sub (' ', '_', taxon),marker),'w') as f:
        f.write(fastas)


Lasioglossum zonulum
Sphecodes Sphecodes_sp
(ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='www.boldsystems.org', port=80): Read timed out.",),), 'Request timed out; does taxon exist?')
Halictus scabiosae
Lasioglossum fulvicorne
Sphecodes geoffrellus
Lasioglossum villosulum
Lasioglossum albipes
Lasioglossum pauxillum
Halictus crenicornis
Halictus fulvipes
Lasioglossum lativentre
Lasioglossum malachurum
Lasioglossum interruptum
Lasioglossum laevigatum
Halictus tumulorum
Halictus Halictus_sp
(ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='www.boldsystems.org', port=80): Read timed out.",),), 'Request timed out; does taxon exist?')
Lasioglossum calceatum
Lasioglossum mediterraneum
Lasioglossum morio
Halictus maculatus
Halictus pollinosus
Halictus rubicundus
Lasioglossum leucozonium
Lasioglossum limbellum
Lasioglossum marginatum
Lasioglossum nigripes
Lasioglossum laticeps


In [ ]:
pairs = zip(headers, seqs)

conseqs = SequenceCollection([DNASequence(seq,label) for (label, seq) in pairs])


In [ ]:
con_aln = muscle_align(conseqs.to_fasta())

In [ ]:
with open('{0}.{1}.{2}.fasta'.format('halictid',marker,'consensus'),'w') as f:
    f.write(conseqs.to_fasta())
with open('{0}.{1}.{2}.fasta'.format('halictid',marker,'consensus_align'),'w') as f:
    f.write(con_aln[0])